<a href="https://colab.research.google.com/github/hassanfv/hfv_GPU/blob/main/acc_sph_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
%%writefile test.cu
#include <iostream>  // iostream, fstream, cmath, string, vector, sstream.
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
using namespace std;

const int N = 131504;
const float my_pi = 3.141592f;
const float visc_alpha = 1.0f;

__global__ void acc_sph(float *x, float *y, float *z, float *vx, float *vy, float *vz,
                        float *h, float *c, float *rho, float *divV, float *curlV,
                        float *mass, float *P, float *ax, float *ay, float *az){

  int i = threadIdx.x + blockIdx.x * blockDim.x;

  if(i < N){

    float dx, dy, dz, rr, hij, q, sig, hij5, gWx, gWy, gWz, nW;
    float vxij, vyij, vzij, wij, vij_rij, vij_sig, rhoij, PIij, fi, fj, fij;
    float axt = 0.0f; float ayt = 0.0f; float azt = 0.0f;

    for(int j = 0; j < N; j++){
      dx = x[i] - x[j];
      dy = y[i] - y[j];
      dz = z[i] - z[j];

      rr = sqrt(dx*dx + dy*dy + dz*dz);

      hij = 0.5f * (h[i] + h[j]);

      if(rr < 2.0f*hij){

        nW = 0.0f; gWx = 0.0f; gWy = 0.0f; gWz = 0.0f;
        sig = 1.0f/my_pi;
        hij5 = hij*hij*hij*hij*hij;
        q = rr/hij;
        
        if(q <= 1.0f){
          nW = sig/hij5 * (-3.0f + (9.0f/4.0f) * q);
          gWx = nW * dx;
          gWy = nW * dy;
          gWz = nW * dz;
        }

        if((q > 1.0f) && (q <=2.0f)){
          nW = -3.0f*sig/(4.0f*hij5) * (2.0f - q)*(2.0f - q) / (q+1e-10);
          gWx = nW * dx;
          gWy = nW * dy;
          gWz = nW * dz;
        }

        //-------- PIij ---------
        vxij = vx[i] - vx[j];
        vyij = vy[i] - vy[j];
        vzij = vz[i] - vz[j];

        vij_rij = vxij*dx + vyij*dy + vzij*dz;

        float cij = 0.5f * (c[i] + c[j]);

        wij = vij_rij / (rr+1e-5);
        vij_sig = c[i] + c[j] - 3.0f * wij;
        rhoij = 0.5f * (rho[i] + rho[j]);

        PIij = 0.0f;
        if(vij_rij <= 0.0f){

          PIij = -0.5f * visc_alpha * vij_sig * wij / rhoij;

          //------- Shear-viscosity correction -------
          fi = divV[i]/(divV[i] + curlV[i] + 0.0001*c[i]/h[i]);
          fj = divV[j]/(divV[j] + curlV[j] + 0.0001*c[j]/h[j]);
          fij = 0.5f * (fi + fj);
          PIij = fij * PIij;
          //------- End of Shear-visc. correction -----
        }

        axt -= mass[j] * (P[i]/rho[i]/rho[i] + P[j]/rho[j]/rho[j] + PIij) * gWx;
        ayt -= mass[j] * (P[i]/rho[i]/rho[i] + P[j]/rho[j]/rho[j] + PIij) * gWy;
        azt -= mass[j] * (P[i]/rho[i]/rho[i] + P[j]/rho[j]/rho[j] + PIij) * gWz;
      }
    }
    ax[i] = axt;
    ay[i] = ayt;
    az[i] = azt;
  }
}


int main(){

  // Reading Hydra file.
  string fname = "Hydra_130k.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  float *x,*y,*z, *vx,*vy,*vz, *rho, *P, *c, *h, *mass, *divV, *curlV;
  float *d_x,*d_y,*d_z, *d_vx,*d_vy,*d_vz, *d_rho, *d_P, *d_c, *d_h;
  float *d_mass, *d_divV, *d_curlV, *ax, *ay, *az, *d_ax, *d_ay, *d_az;

  x = new float[N];
  y = new float[N];
  z = new float[N];

  vx = new float[N];
  vy = new float[N];
  vz = new float[N];

  ax = new float[N];
  ay = new float[N];
  az = new float[N];

  rho = new float[N];
  P = new float[N];
  c = new float[N];
  h = new float[N];
  mass = new float[N];
  divV = new float[N];
  curlV = new float[N];

  // 0  1  2  3   4   5    6   7  8  9  10
  // x, y, z, vx, vy, vz, rho, P, c, h, m.

  for(int i=0; i<N; i++){

    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    vx[i] = stof(content[i][3]);
    vy[i] = stof(content[i][4]);
    vz[i] = stof(content[i][5]);

    rho[i] = stof(content[i][6]);
    P[i] = stof(content[i][7]);
    c[i] = stof(content[i][8]);
    h[i] = stof(content[i][9]);
    mass[i] = stof(content[i][10]);

    divV[i] = stof(content[i][11]);
    curlV[i] = stof(content[i][12]);

    ax[i] = 0.0f;
    ay[i] = 0.0f;
    az[i] = 0.0f;

  }

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_vx, N*sizeof(float));
  cudaMalloc(&d_vy, N*sizeof(float));
  cudaMalloc(&d_vz, N*sizeof(float));

  cudaMalloc(&d_ax, N*sizeof(float));
  cudaMalloc(&d_ay, N*sizeof(float));
  cudaMalloc(&d_az, N*sizeof(float));

  cudaMalloc(&d_rho, N*sizeof(float));
  cudaMalloc(&d_P, N*sizeof(float));
  cudaMalloc(&d_c, N*sizeof(float));
  cudaMalloc(&d_h, N*sizeof(float));
  cudaMalloc(&d_mass, N*sizeof(float));
  cudaMalloc(&d_divV, N*sizeof(float));
  cudaMalloc(&d_curlV, N*sizeof(float));

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_vx, vx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vy, vy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vz, vz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_ax, ax, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_ay, ay, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_az, az, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_rho, rho, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_P, P, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_c, c, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_h, h, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_divV, divV, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_curlV, curlV, N*sizeof(float), cudaMemcpyHostToDevice);

  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  acc_sph<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, d_h, d_c, d_rho,
                                   d_divV, d_curlV, d_mass, d_P, d_ax, d_ay, d_az);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // Copy from Device to Host.
  cudaMemcpy(ax, d_ax, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(ay, d_ay, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(az, d_az, N*sizeof(float), cudaMemcpyDeviceToHost);

  // Output to a file
  ofstream outfile("div_curlV_from_cpp_130k.csv");
  if(outfile.is_open()){
    for(int i = 0; i < N; i++){
      outfile << ax[i] << "," << ay[i] << "," << az[i] << endl;
    }
  }else cout << "Unable to open file !";


  // visual check.
  for(int i = N-10; i < N; i++){
    cout << ax[i] << "," << ay[i] << "," << az[i] << endl;
  }

  // Free memory.
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_vx);
  cudaFree(d_vy);
  cudaFree(d_vz);

  cudaFree(d_ax);
  cudaFree(d_ay);
  cudaFree(d_az);

  cudaFree(d_rho);
  cudaFree(d_P);
  cudaFree(d_c);
  cudaFree(d_h);
  cudaFree(d_mass);
  cudaFree(d_divV);
  cudaFree(d_curlV);

  delete[] x;
  delete[] y;
  delete[] z;

  delete[] vx;
  delete[] vy;
  delete[] vz;

  delete[] ax;
  delete[] ay;
  delete[] az;

  delete[] rho;
  delete[] P;
  delete[] c;
  delete[] h;
  delete[] mass;
  delete[] divV;
  delete[] curlV;

}

Overwriting test.cu


In [52]:
%%shell
nvcc test.cu -o test

In [53]:
%%shell
./test

5.26792,-9.48758,0.730126
-1.83794,0.28572,-3.64801
-3.51097,-12.3785,12.7718
-3.9401,12.7879,0.634667
-9.08467,-5.85098,3.54592
4.42849,-0.714199,2.6781
-3.55973,1.06215,2.88004
0.604275,-3.54694,0.875054
-1.20097,4.04645,6.25638
-6.05815,-2.55019,1.68031


In [54]:
%%shell
nvprof ./test

==855== NVPROF is profiling process 855, command: ./test
5.26792,-9.48758,0.730126
-1.83794,0.28572,-3.64801
-3.51097,-12.3785,12.7718
-3.9401,12.7879,0.634667
-9.08467,-5.85098,3.54592
4.42849,-0.714199,2.6781
-3.55973,1.06215,2.88004
0.604275,-3.54694,0.875054
-1.20097,4.04645,6.25638
-6.05815,-2.55019,1.68031
==855== Profiling application: ./test
==855== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.75%  341.50ms         1  341.50ms  341.50ms  341.50ms  acc_sph(float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*)
                    0.21%  724.53us        16  45.283us  45.119us  45.823us  [CUDA memcpy HtoD]
                    0.04%  124.73us         3  41.577us  41.471us  41.631us  [CUDA memcpy DtoH]
      API calls:   50.88%  341.54ms         1  341.54ms  341.54ms  341.54ms  cudaDeviceSynchronize
                   48.44%  325.13ms 